In [5]:
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import re


# 1) Carpeta donde están los archivos (ajusta la ruta)
carpeta = Path("data/CEFALOSPORINAS base de datos ADISINSIGHT")  # o la ruta absoluta

# 2) Recolecta todos los .xls y .xlsx (evita subcarpetas tipo 'processed' si quieres)
files = [p for p in carpeta.glob("*.xls*") if p.is_file()]

dfs, errores = [], []

for f in files:
    try:
        df_tmp = pd.read_excel(f)  # si usas .xlsx puro, puedes forzar engine="openpyxl"
        # 3) Columna con el tipo de antibiótico desde el nombre del archivo
        df_tmp["Antibiotico"] = f.stem.strip()  # .stem = nombre sin extensión
        dfs.append(df_tmp)
    except Exception as e:
        errores.append((f.name, str(e)))

# 4) Concatena (alinea por nombre de columna)
df_all = pd.concat(dfs, ignore_index=True, sort=False)

# 5) (Opcional) poner 'Antibiotico' como primera columna
cols = ["Antibiotico"] + [c for c in df_all.columns if c != "Antibiotico"]
df_all = df_all[cols]

# 6) (Opcional) guardar
# df_all.to_parquet(carpeta / "processed" / "cefalo_concatenado.parquet", index=False)
# o Excel:
# df_all.to_excel(carpeta / "processed" / "cefalo_concatenado.xlsx", index=False)

# 7) (Opcional) revisar qué antibióticos se cargaron
print(sorted(df_all["Antibiotico"].unique()))
print("Errores:", errores)


['CEFACLOR', 'CEFALEXINA', 'CEFAZOLINA', 'CEFEPIME', 'CEFIXIMA', 'CEFOPERAZONA', 'CEFOTAXIMA', 'CEFOTETAN', 'CEFOXITINA', 'CEFTAROLINA', 'CEFTAZIDIMA', 'CEFTAZIDIMA CON AVIBACTAM', 'CEFTIBUTEN', 'CEFTOLOZANO', 'CEFTRIAXONA', 'CEFUROXIMA']
Errores: []


In [6]:
df_all

,Antibiotico,Report Id,Safety Report Name,Record Type,Release Date,Drugs,Adverse Events,Mechanism of Action,Number of Cases,Serious,...,Reporter Country,Source Country,Country of publication,Drug Class,Age Group,Route Of Administration,Treatment,Special Situation,References,URL
0,CEFACLOR,803595494,"Multiple drugs - Exposure during pregnancy, of...",Special Situation Case Report,09 Sep 2021,"Azithromycin adverse reactions, Beclometasone ...",NaN,NaN,2.0,No,...,China,NaN,USA,"Acetamides , Anti-inflammatories , Antiallergi...",NaN,"Intravenous, Oral, Topical",NaN,NaN,"1. Fan C, Lei D, Fang C, Li C, Wang M, Liu Y, ...",https://adis.springer.com/drugsafety/803595494...
1,CEFACLOR,803589340,Multiple drugs - Drug-induced skin eruption an...,ADR Case Report,16 Aug 2021,"Aztreonam adverse reactions (Serious), Cefaclo...","Drug eruptions drug-induced, Skin eruptions dr...",NaN,1.0,Yes,...,Japan,NaN,France,"Anti-infectives , Antibacterials , Antifungals...",Elderly,Oral,NaN,NaN,"1. Kumano Y, Nagai A, Sugiura K. Erythro-papul...",https://adis.springer.com/drugsafety/803589340...
2,CEFACLOR,803582224,Risk of delirium with ampicillin/sulbactam and...,News,20 Jul 2021,"Ampicillin adverse reactions, Ampicillin/sulba...",Delirium drug-induced,NaN,NaN,No,...,NaN,USA,New Zealand,"Acetamides , Amides , Anti-infectives , Anti-i...",NaN,NaN,NaN,NaN,"1. Teng C, Frei CR. Delirium Associations with...",https://adis.springer.com/drugsafety/803582224...
3,CEFACLOR,803578298,Multiple drugs - Drug hypersensitivity reactio...,ADR Case Report,16 Jul 2021,"Amoxicillin adverse reactions (Serious), Ampic...","Anaphylaxis drug-induced, Angioedema drug-indu...",NaN,86.0,Yes,...,France,NaN,USA,"Anti-infectives , Antibacterials , Cephalospor...",Adolescents,NaN,NaN,NaN,"1. Touati N, Cardoso B, Delpuech M, Bazire R, ...",https://adis.springer.com/drugsafety/803578298...
4,CEFACLOR,803562921,Multiple drugs - Off label use and maternal ex...,Special Situation Case Report,14 May 2021,"Azithromycin adverse reactions, Cefaclor adver...",NaN,NaN,10.0,No,...,China,NaN,USA,"Acetamides , Anti-infectives , Anti-inflammato...",NaN,NaN,NaN,NaN,"1. Fan C, Guo Y, Qu P, Wang S, Wang M, Yuan J,...",https://adis.springer.com/drugsafety/803562921...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179,CEFUROXIMA,800058971,Cefuroxime/gentamicin - Toxicity in cataract s...,ADR Case Report,20 Feb 1991,"Cefuroxime adverse reactions, Gentamicin adver...",Eye disorders drug-induced,NaN,NaN,No,...,England,NaN,England,"Aminoglycosides , Anti-infectives , Antibacter...",NaN,Ophthalmic,NaN,NaN,"1. Jenkins CDG, McDonnell PJ, Spalton DJ. Rand...",https://adis.springer.com/drugsafety/800058971...
3180,CEFUROXIMA,800508930,Cefuroxime - First report of fever: case report,ADR Case Report,08 Jun 1989,Cefuroxime adverse reactions,Fever drug-induced,NaN,1.0,No,...,Unknown,NaN,Spain,"Anti-infectives adverse reactions, Antibacteri...",Children,"Intramuscular, Intravenous",NaN,NaN,"1. Martinez Abad M, Delgado Gomis F, Morales O...",https://adis.springer.com/drugsafety/800508930...
3181,CEFUROXIMA,800503108,Cefuroxime - Hematuria: case report,ADR Case Report,14 Sep 1988,Cefuroxime,"Blood coagulation disorders drug-induced, Haem...",NaN,1.0,No,...,Unknown,NaN,England,"Anti-infectives , Antibacterials , Cephalospor...",NaN,Intravenous,NaN,NaN,"1. Lewis D, Allanson H. Cefuroxime and hematur...",https://adis.springer.com/drugsafety/800503108...
3182,CEFUROXIMA,800505454,Cefuroxime metronidazole - Postoperative bleed...,ADR Case Report,12 Apr 1986,"Cefuroxime adverse reactions, Cefuroxime drug ...","Blood coagulation disorders drug-induced, Peri...",NaN,NaN,No,...,Unknown,NaN,England,"Anti-infectives adverse reactions, Anti-infect...",NaN,Intravenous,NaN,NaN,"1. Kingston RD, Duthie J. Original Article Tit...",https://adis.springer.com/drugsafety/800505454...


In [7]:
def solo_un_paciente(df):
    return  df[df['Number of Cases']==1]
    
#ejemplo el df 1 que utlizaremos para hacer la limpieza en este nb 

df = solo_un_paciente(df_all)

# DRUG CLASSES

In [8]:
COL = 'Drug Class'  

# 1) Normalizar cada celda pero SIN romper conceptos internos
def normalize_phrase(p: str) -> str:
    s = p.upper()
    s = s.replace('_', ' ').replace('-', ' ')       # joiners -> espacio
    s = s.replace(' ADVERSE REACTIONS', '')         # ruido común
    s = s.replace(' (SERIOUS)', '') 
    s = re.sub(r'\s+', ' ', s).strip()              # espacios extra
    return s

def parse_classes(cell) -> list:
    if pd.isna(cell) or str(cell).strip()=='':
        return []
    # REGLA: solo separamos por coma (concepto por concepto)
    raw = [c.strip() for c in str(cell).split(',') if c.strip()]
    classes = [normalize_phrase(c) for c in raw]
    # deduplicar por registro y ordenar para estabilidad
    return sorted(set(classes))

# lista de clases por registro (frases)
df['Drug_Class_list'] = df[COL].apply(parse_classes)

# 2) Conteo de clases únicas (frases)
class_counts = (
    pd.Series([c for lst in df['Drug_Class_list'] for c in lst])
      .value_counts()
)
print(class_counts.head(30)) # quitar esto para poder solo tener el df sin print 



# 3) Dummies por clase (frase completa)
mlb = MultiLabelBinarizer()
drug_dummies = pd.DataFrame(
    mlb.fit_transform(df['Drug_Class_list']),
    columns=[c.replace(' ', '_') for c in mlb.classes_],  # columnas limpias
    index=df.index
)

df = pd.concat([df, drug_dummies], axis=1)

ANTIBACTERIALS                      2013
CEPHALOSPORINS                      1964
SMALL MOLECULES                     1689
ANTI INFECTIVES                      628
ANTI INFLAMMATORIES                  316
EYE DISORDER THERAPIES               314
ANTINEOPLASTICS                      271
PEPTIDE ANTIBIOTICS                  267
SKIN DISORDER THERAPIES              263
MACROLIDES                           261
PENICILLINS                          259
ANTIVIRALS                           243
GLYCOPEPTIDES                        235
ANTIRHEUMATICS                       226
GLUCOCORTICOIDS                      189
ANTIMALARIALS                        187
ETHANOLAMINES                        181
AMIDES                               174
ANTIPROTOZOALS                       173
PYRROLIDINES                         163
THIENAMYCINS                         163
CHLOROBENZENES                       157
ANTIALLERGICS                        152
DISEASE MODIFYING ANTIRHEUMATICS     151
PIPERAZINES     

C:\Users\narro\AppData\Local\Temp\ipykernel_568844\2995967544.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Drug_Class_list'] = df[COL].apply(parse_classes)


In [9]:
df

,Antibiotico,Report Id,Safety Report Name,Record Type,Release Date,Drugs,Adverse Events,Mechanism of Action,Number of Cases,Serious,...,VASODILATORS,VASOPRESSINS,VASOPRESSORS,VASOPRESSORS_THERAPEUTIC_USE,VESTIBULAR_DISORDER_THERAPIES,VINCA_ALKALOIDS,VITAMIN_B12_ANALOGUES,VITAMIN_D_ANALOGUES,VITAMINS,XANTHINES
1,CEFACLOR,803589340,Multiple drugs - Drug-induced skin eruption an...,ADR Case Report,16 Aug 2021,"Aztreonam adverse reactions (Serious), Cefaclo...","Drug eruptions drug-induced, Skin eruptions dr...",NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,0,0
5,CEFACLOR,803562657,Antibacterials/ganciclovir/ibuprofen - Lack of...,ADR Case Report,14 May 2021,"Cefaclor adverse reactions (Serious), Cefaclor...",Lack of efficacy in critical conditions drug-i...,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,0,0
11,CEFACLOR,803476613,Amoxicillin/clavulanic acid/cefaclor - Toxic e...,ADR Case Report,22 May 2020,"Amoxicillin adverse reactions (Serious), Amoxi...",Toxic epidermal necrolysis drug-induced,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,0,0
12,CEFACLOR,803475361,Cefaclor - Nausea: case report,ADR Case Report,16 May 2020,Cefaclor adverse reactions,Nausea drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,0,0
14,CEFACLOR,803353779,Cefaclor/nonsteroidal anti-inflammatories - DR...,ADR Case Report,26 Oct 2018,Cefaclor adverse reactions (Serious),"Cardiogenic shock drug-induced, DRESS syndrome...",NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,CEFUROXIMA,800217085,Cefuroxime/ceftazidime - Agranulocytosis in an...,ADR Case Report,01 Oct 1993,"Ceftazidime adverse reactions (Serious), Cefur...",Blood dyscrasias drug-induced,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,0,0
3178,CEFUROXIMA,800170784,Cefuroxime - Delayed hypersensitivity: case re...,ADR Case Report,24 Dec 1992,Cefuroxime adverse reactions,Drug hypersensitivity drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,0,0
3180,CEFUROXIMA,800508930,Cefuroxime - First report of fever: case report,ADR Case Report,08 Jun 1989,Cefuroxime adverse reactions,Fever drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,0,0
3181,CEFUROXIMA,800503108,Cefuroxime - Hematuria: case report,ADR Case Report,14 Sep 1988,Cefuroxime,"Blood coagulation disorders drug-induced, Haem...",NaN,1.0,No,...,0,0,0,0,0,0,0,0,0,0


In [10]:
drug_cols = list(drug_dummies.columns)

df['n_drug_classes'] = df[drug_cols].sum(axis=1)
df['polytherapy']    = (df['n_drug_classes'] > 1).astype(int)
df

,Antibiotico,Report Id,Safety Report Name,Record Type,Release Date,Drugs,Adverse Events,Mechanism of Action,Number of Cases,Serious,...,VASOPRESSORS,VASOPRESSORS_THERAPEUTIC_USE,VESTIBULAR_DISORDER_THERAPIES,VINCA_ALKALOIDS,VITAMIN_B12_ANALOGUES,VITAMIN_D_ANALOGUES,VITAMINS,XANTHINES,n_drug_classes,polytherapy
1,CEFACLOR,803589340,Multiple drugs - Drug-induced skin eruption an...,ADR Case Report,16 Aug 2021,"Aztreonam adverse reactions (Serious), Cefaclo...","Drug eruptions drug-induced, Skin eruptions dr...",NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,11,1
5,CEFACLOR,803562657,Antibacterials/ganciclovir/ibuprofen - Lack of...,ADR Case Report,14 May 2021,"Cefaclor adverse reactions (Serious), Cefaclor...",Lack of efficacy in critical conditions drug-i...,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,13,1
11,CEFACLOR,803476613,Amoxicillin/clavulanic acid/cefaclor - Toxic e...,ADR Case Report,22 May 2020,"Amoxicillin adverse reactions (Serious), Amoxi...",Toxic epidermal necrolysis drug-induced,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,4,1
12,CEFACLOR,803475361,Cefaclor - Nausea: case report,ADR Case Report,16 May 2020,Cefaclor adverse reactions,Nausea drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,2,1
14,CEFACLOR,803353779,Cefaclor/nonsteroidal anti-inflammatories - DR...,ADR Case Report,26 Oct 2018,Cefaclor adverse reactions (Serious),"Cardiogenic shock drug-induced, DRESS syndrome...",NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,CEFUROXIMA,800217085,Cefuroxime/ceftazidime - Agranulocytosis in an...,ADR Case Report,01 Oct 1993,"Ceftazidime adverse reactions (Serious), Cefur...",Blood dyscrasias drug-induced,NaN,1.0,Yes,...,0,0,0,0,0,0,0,0,3,1
3178,CEFUROXIMA,800170784,Cefuroxime - Delayed hypersensitivity: case re...,ADR Case Report,24 Dec 1992,Cefuroxime adverse reactions,Drug hypersensitivity drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,2,1
3180,CEFUROXIMA,800508930,Cefuroxime - First report of fever: case report,ADR Case Report,08 Jun 1989,Cefuroxime adverse reactions,Fever drug-induced,NaN,1.0,No,...,0,0,0,0,0,0,0,0,3,1
3181,CEFUROXIMA,800503108,Cefuroxime - Hematuria: case report,ADR Case Report,14 Sep 1988,Cefuroxime,"Blood coagulation disorders drug-induced, Haem...",NaN,1.0,No,...,0,0,0,0,0,0,0,0,3,1


In [11]:
df['Narrative_clean'] = df['Narrative and comments'].str.lower()


def extract_age(text):
    if not isinstance(text, str):
        return None
    # Buscar "X-year-old"
    year_match = re.search(r'(\d+)\s*[-]?\s*year[- ]?old', text)
    if year_match:
        return int(year_match.group(1))
    # Buscar "X-month-old"
    month_match = re.search(r'(\d+)\s*[-]?\s*month[- ]?old', text)
    if month_match:
        return round(int(month_match.group(1)) / 12, 2)
    return None

df['Age'] = df['Narrative_clean'].apply(extract_age)

In [12]:
female_keywords = ['female', 'woman', 'girl', 'lady']
male_keywords = ['male', 'man', 'boy', 'gentleman']

def extract_gender(text):
    if not isinstance(text, str):
        return 'non-specified'
    for word in female_keywords:
        if word in text:
            return 'female'
    for word in male_keywords:
        if word in text:
            return 'male'
    return 'non-specified'

df['Gender'] = df['Narrative_clean'].apply(extract_gender)

In [14]:
import numpy as np
# --- 1) Extracción de temperatura numérica ---
# Captura formas comunes: '39°C', '39 °c', 'temp 39.2 c', 'temperature 102 f', 't=38.5°c'
cel_pat = re.compile(r'(?:(?:temp(?:erature)?|t)[^\d]{0,5})?(\d{1,2}(?:[.,]\d{1,2})?)\s*°?\s*(?:c|celsius)\b')
fah_pat = re.compile(r'(?:(?:temp(?:erature)?|t)[^\d]{0,5})?(\d{2,3}(?:[.,]\d{1,2})?)\s*°?\s*(?:f|fahrenheit)\b')
# Casos sin unidad pero con "°" y rango razonable (25–45)
bare_deg_pat = re.compile(r'(\d{1,2}(?:[.,]\d{1,2})?)\s*°\b')

def _to_float(num_str):
    try:
        return float(num_str.replace(',', '.'))
    except Exception:
        return None

def extract_temp_c(text: str):
    if not isinstance(text, str) or not text:
        return np.nan

    # 1) Celsius explícito
    m = cel_pat.search(text)
    if m:
        val = _to_float(m.group(1))
        # Filtro de plausibilidad humana
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    # 2) Fahrenheit explícito -> convertir
    m = fah_pat.search(text)
    if m:
        f = _to_float(m.group(1))
        if f is not None and 80 <= f <= 113:  # ~26.7–45°C
            c = (f - 32) * 5/9
            return round(c, 2)

    # 3) Número con símbolo ° sin unidad, validar rango humano
    m = bare_deg_pat.search(text)
    if m:
        val = _to_float(m.group(1))
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    # 4) Patrones tipo "38-39°c" -> tomar el primero
    range_cel = re.search(r'(\d{1,2}(?:[.,]\d{1,2})?)\s*[-–]\s*\d{1,2}(?:[.,]\d{1,2})?\s*°?\s*(?:c|celsius)\b', text)
    if range_cel:
        val = _to_float(range_cel.group(1))
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    return np.nan

df['Temp_C'] = df['Narrative_clean'].apply(extract_temp_c)

# --- 2) Clasificación clínica ---
# Umbrales (ajústalos si quieres):
# <35.0 -> hypothermia ; 35.0–37.4 -> normal ; >=37.5 -> fever
def classify_temp_from_value(c):
    if pd.isna(c):
        return None
    if c < 35.0:
        return 'hypothermia'
    if c >= 37.5:
        return 'fever'
    return 'normal'

# Palabras clave cuando no hay número
# Nota: primero checamos negaciones como "afebrile", "no fever"
normal_kw = [
    'afebrile', 'apyrexial', 'apyrexic', 'no fever', 'without fever', 'defervesc',
]
fever_kw = [
    'fever', 'febrile', 'pyrexia', 'high temperature', 'high-grade fever', 'low-grade fever'
]
hypo_kw = ['hypothermia', 'hypothermic']

def classify_temp_from_text(text: str):
    if not isinstance(text, str) or not text:
        return 'non-specified'
    # normal (negaciones primero)
    for w in normal_kw:
        if w in text:
            return 'normal'
    # hypothermia
    for w in hypo_kw:
        if w in text:
            return 'hypothermia'
    # fever
    for w in fever_kw:
        if w in text:
            return 'fever'
    return 'non-specified'

def final_temp_category(row):
    c = row['Temp_C']
    if pd.notna(c):
        return classify_temp_from_value(c)
    # sin número: inferir por texto
    return classify_temp_from_text(row['Narrative_clean'])

df['Temp_Category'] = df.apply(final_temp_category, axis=1)

In [16]:

# --- Listas de palabras clave por categoría ---
kw = {
    "fungus": [
        r"\bfungal\b", r"\bmycosis\b", r"\bonychomycosis\b", r"\botomycosis\b",
        r"\bcandid(?:a|iasis)\b", r"\baspergillus\b", r"\bcryptococcus\b",
        r"\bhistoplasma\b", r"\bblastomyces\b", r"\bcoccidioides\b",
        r"\bdermatophyte\b", r"\btinea\b", r"\bmucor(?:mycosis)?\b",
        r"\bsporothrix\b"
    ],
    "parasite": [
        r"\bparasite|parasitic|parasitism\b", r"\bprotozoa(?:n|l)?\b", r"\bhelminth\b",
        r"\bmalaria\b", r"\bplasmodium\b", r"\bgiardi(?:a|asis)\b",
        r"\bamoeb(?:a|iasis)\b", r"\bentamoeba\b", r"\btrichomonas\b",
        r"\btoxoplasma(?:osis)?\b", r"\bleishmania(?:sis)?\b",
        r"\btaeni(?:a|asis)\b", r"\bechinococcus\b", r"\bschistosom(?:a|iasis)\b",
        r"\bstrongyloides\b", r"\bscabies\b", r"\bpediculosis\b", r"\bpinworm\b", r"\bhookworm\b"
    ],
    "virus": [
        r"\bviral\b", r"\bvir(?:us|al) infection\b", r"\binfluenza\b", r"\bflu\b",
        r"\bvaricella\b", r"\bherpes(?:\s(simplex|zoster))?\b", r"\brsv\b",
        r"\bsars[- ]?cov[- ]?2\b", r"\bcovid[- ]?19\b", r"\bhiv\b",
        r"\bhbv\b", r"\bhcv\b", r"\bhpv\b", r"\bebv\b", r"\bcmv\b",
        r"\bnorovirus\b", r"\badenovirus\b", r"\brotavirus\b", r"\benterovirus\b",
        r"\bc(orona)?virus\b"
    ],
    "bacteria": [
        r"\bbacterial\b", r"\bbacteremia\b", r"\bcellulitis\b", r"\bpyelonephritis\b",
        r"\bmrsa\b", r"\bmssa\b", r"\bstaph(?:ylococcus)?\b", r"\bstrept(?:ococcus|ococcal|ococcus)\b",
        r"\bpseudomonas\b", r"\bescherichia\s*coli\b|\becoli\b", r"\benterobacter\b",
        r"\bklebs(?:iella)?\b", r"\bproteus\b", r"\bacinetobacter\b", r"\bserratia\b",
        r"\bclostrid(?:ium|ioides)\b", r"\blisteria|listeria\b", r"\bmycobacter(?:ium|ia)\b",
        r"\bgram[- ]?(positive|negative)\b", r"\buti\b", r"\burninary tract infection\b"
    ],
}

# Compilar patrones por categoría para rendimiento
compiled = {cat: re.compile("|".join(pats)) for cat, pats in kw.items()}

def detect_infection_types(text: str):
    if not isinstance(text, str) or not text:
        return set()
    found = set()
    for cat, pat in compiled.items():
        if pat.search(text):
            found.add(cat)
    return found

def collapse_to_category(found: set):
    if not found:
        return "non-specified"
    if len(found) == 1:
        return list(found)[0]
    return "mixed"

# Columna categórica principal
df['Infection_Type'] = df['Narrative_clean'].apply(lambda t: collapse_to_category(detect_infection_types(t)))

# (Opcional) one-hot por categoría, por si te sirve para análisis
#for cat in kw.keys():
#   df[f'is_{cat}'] = df['Narrative_clean'].apply(lambda t: 1 if compiled[cat].search(t or "") else 0)

# Inspección rápida:
# df['Infection_Type'].value_counts()
# df[['Infection_Type', 'is_fungus','is_parasite','is_virus','is_bacteria']].head()


In [17]:
import re
import pandas as pd
import numpy as np

# --- 1) Lista de cefalosporinas (incluye variantes ES/EN) ---
cephalosporins = [
    r"cefaclor",
    r"cefalexina|cephalexin",
    r"cefazolina|cefazolin",
    r"cefepime",
    r"cefixima|cefixime",
    r"cefoperazona|cefoperazone",
    r"cefotaxima|cefotaxime",
    r"cefotetan",
    r"cefoxitina|cefoxitin",
    r"ceftarolina|ceftaroline",
    r"ceftazidima(?:\s+con\s+avibactam)?|ceftazidime(?:\s+with\s+avibactam)?|ceftazidime\s*/\s*avibactam",
    r"ceftibuten",
    r"ceftolozano|ceftolozane",
    r"ceftriaxona|ceftriaxone",
    r"cefuroxima|cefuroxime",
]
drug_pat = re.compile(r"(" + r"|".join(cephalosporins) + r")\b", flags=re.I)

# --- 2) Patrones de rutas de administración ---
route_map = {
    "oral": [
        r"\boral\b", r"\bpo\b", r"\bper\s*os\b", r"\bby\s*mouth\b", r"\bvo\b"
    ],
    "iv": [
        r"\bintraven(?:ous|a)\b", r"\biv\b", r"\biv\s*(bolus|infusion|drip)\b",
        r"\bpre[- ]?op(?:erative)?\s+prophylaxis\b"  # suele ser IV en quirófano
    ],
    "im": [r"\bintramuscular\b", r"\bim\b"],
    "sc": [r"\bsubcut(?:aneous|a)\b", r"\bsc\b", r"\bsq\b"],
    "topical": [r"\btopical\b", r"\bointment\b", r"\bcream\b", r"\bgel\b"],
    "ophthalmic": [r"\bophthalmic\b", r"\beye\s*drops?\b"],
    "otic": [r"\botic\b", r"\bear\s*drops?\b"],
    "intrathecal": [r"\bintrathecal\b", r"\bit\b"],
    "inhaled": [r"\binhal(?:ed|ation)\b", r"\bnebuliz(?:ed|er|ación)\b"],
    "rectal": [r"\brectal\b", r"\bpr\b", r"\bsuppositor(y|ies)\b"],
    "vaginal": [r"\bvaginal\b", r"\bpv\b"],
    "transdermal": [r"\btransdermal\b", r"\bpatch\b"],
    "intraperitoneal": [r"\bintraperitoneal\b", r"\bip\b"],
    "intraarticular": [r"\bintra[- ]?articular\b", r"\bia\b"],
}
compiled_routes = {k: re.compile("|".join(v)) for k, v in route_map.items()}

def find_routes(text: str):
    """Devuelve set de rutas encontradas y origen ('near-drug' o 'global')."""
    if not isinstance(text, str) or not text:
        return set(), None

    # 1) buscar menciones de drogas y revisar ventana cercana
    routes = set()
    for m in drug_pat.finditer(text):
        start, end = m.span()
        window = text[max(0, start-140): min(len(text), end+140)]
        for route, pat in compiled_routes.items():
            if pat.search(window):
                routes.add(route)

    if routes:
        return routes, "near-drug"

    # 2) búsqueda global si no se halló cerca del fármaco
    for route, pat in compiled_routes.items():
        if pat.search(text):
            routes.add(route)
    if routes:
        return routes, "global"

    return set(), None

# Para textos que dicen "was given" sin ruta pero mencionan "tablet/capsule/iv line"
infer_map = {
    "oral": re.compile(r"\btablet|capsule|suspension\b"),
    "iv": re.compile(r"\bline|cannula|infus(?:ion|ed)\b"),
}
def infer_route(text: str):
    if not isinstance(text, str) or not text:
        return None
    for r, pat in infer_map.items():
        if pat.search(text):
            return r
    return None

def collapse_routes(routes: set):
    if not routes:
        return "non-specified"
    if len(routes) == 1:
        return list(routes)[0]
    return "multiple"

def extract_admin_route(text: str):
    routes, origin = find_routes(text)
    confidence = None
    if routes:
        confidence = "high" if origin == "near-drug" else "medium"
    else:
        # intento de inferencia débil
        guessed = infer_route(text)
        if guessed:
            routes = {guessed}
            confidence = "low"
    return collapse_routes(routes), origin or ("inferred" if routes else None), confidence

# --- columnas derivadas ---
tmp = df['Narrative_clean'].apply(extract_admin_route)
df[['Admin_Route', 'Admin_Route_source', 'Admin_Route_confidence']] = pd.DataFrame(tmp.tolist(), index=df.index)

# (Opcional) indicadores binarios por ruta para análisis
#for r in route_map.keys():
 #   df_nar[f'route_{r}'] = df_nar['Narrative_clean'].apply(lambda t: 1 if compiled_routes[r].search(t or "") else 0)



In [18]:

# --- Palabras clave de convulsiones ---
seizure_kw = [
    r"\bseizure(s)?\b",
    r"\bconvulsion(s)?\b",
    r"\bepileptic\s+(fit|seizure|attack)\b",
    r"\bepilepsy\b",
    r"\bstatus\s+epilepticus\b",
    r"\btonic[- ]?clonic\b",
    r"\bgrand\s+mal\b",
    r"\bfebrile\s+seizure\b",
    r"\babsence\s+seizure\b",
    r"\bmyoclonic\b",
    r"\bclonic\b",
    r"\btonic\b",
    r"\bataque\s+epiléptico\b",
    r"\bcrisis\s+epiléptica\b",
    r"\bcrisis\s+convulsiva\b",
    r"\bconvulsión(es)?\b",
    r"\bconvulsivo\b",
]

# --- Compilar patrón ---
seizure_pat = re.compile("|".join(seizure_kw), flags=re.I)

# --- Crear variable binaria ---
df['Seizure'] = df['Narrative_clean'].apply(lambda t: 1 if isinstance(t, str) and seizure_pat.search(t) else 0)

# --- (Opcional) Verificación ---
# df['Seizure'].value_counts()
# df.loc[df['Seizure'] == 1, ['Report Id', 'Narrative and comments']].head()


In [19]:
pd.set_option('display.max_columns',None)

In [20]:
df

,Antibiotico,Report Id,Safety Report Name,Record Type,Release Date,Drugs,Adverse Events,Mechanism of Action,Number of Cases,Serious,First Report,Overdose,Drug Interaction,Narrative and comments,Reporter Country,Source Country,Country of publication,Drug Class,Age Group,Route Of Administration,Treatment,Special Situation,References,URL,Drug_Class_list,2_PYRIDINYLMETHYLSULFINYLBENZIMIDAZOLES,2_RING_HETEROCYCLIC_COMPOUNDS,3_RING_HETEROCYCLIC_COMPOUNDS,ABORTIFACIENTS,ACETAMIDES,ACETANILIDES,ACETATES,ACETIC_ACIDS,ACYCLIC_ACIDS,ADENINE_NUCLEOTIDES,ADJUVANTS,ALCOHOLS,ALKALINISING_AGENTS,ALKALOIDS,AMIDES,AMINES,AMINO_ACIDS,AMINOBENZOIC_ACIDS,AMINOBUTYRIC_ACIDS,AMINOCYCLITOLS,AMINOGLYCOSIDES,AMINOPHENOLS,AMINOQUINOLINES,AMINOSALICYLIC_ACIDS,ANAESTHETICS,ANALGESICS,ANDROSTADIENES,ANDROSTANOLS,ANILIDES,ANISOLES,ANTHELMINTICS,ANTHRACYCLINES,ANTI_INFECTIVES,ANTI_INFECTIVES_DRUG_INTERACTIONS,ANTI_INFLAMMATORIES,ANTI_ISCHAEMICS,ANTI_TNF_MONOCLONAL_ANTIBODIES,ANTIACNES,ANTIALLERGICS,ANTIANAEMICS,ANTIANDROGENS,ANTIARRHYTHMICS,ANTIASTHMATICS,ANTIBACTERIALS,ANTIBACTERIALS_THERAPEUTIC_USE,ANTIBRONCHITICS,ANTICOAGULANTS,ANTIDEMENTIAS,ANTIDEPRESSANTS,ANTIDIARRHOEALS,ANTIDIURETICS,ANTIDOTES,ANTIEMETICS,ANTIEPILEPTIC_DRUGS,ANTIFIBRINOLYTICS,ANTIFIBROTICS,ANTIFUNGALS,ANTIFUNGALS_THERAPEUTIC_USE,ANTIGLAUCOMAS,ANTIGOUTS,ANTIHAEMORRHAGICS,ANTIHISTAMINES,ANTIHISTAMINES_THERAPEUTIC_USE,ANTIHORMONES,ANTIHYPERCALCAEMICS,ANTIHYPERGLYCAEMICS,ANTIHYPERLIPIDAEMICS,ANTIHYPERTENSIVES,ANTIHYPOGLYCAEMICS,ANTIHYPOTENSIVES,ANTILEPROTICS,ANTIMALARIALS,ANTIMANICS,ANTIMIGRAINES,ANTINEOPLASTICS,ANTIPARASITICS,ANTIPARKINSONIANS,ANTIPLATELETS,ANTIPROTOZOALS,ANTIPRURITICS,ANTIPSORIATICS,ANTIPSYCHOTICS,ANTIPYRETICS,ANTIRETROVIRALS,ANTIRHEUMATICS,ANTISEPTICS,ANTISPASMODICS,ANTITHROMBOTICS,ANTITOXINS,ANTITUBERCULARS,ANTITUSSIVES,ANTIULCERS,ANTIVIRALS,ANXIOLYTICS,ARABINONUCLEOSIDES,AROMATIC_AMINO_ACIDS,ARSENICALS,ARTEMISININS,ATROPINE_DERIVATIVES,AZA_COMPOUNDS,AZABICYCLO_COMPOUNDS,AZETIDINES,AZIDES,AZOLES,BACTERIAL_VACCINES,BARBITURATES,BASIC_AMINO_ACIDS,BEHAVIOURAL_DISORDER_THERAPIES,BENZAMIDES,BENZAMIDINES,BENZENESULFONATES,BENZHYDRYL_COMPOUNDS,BENZIMIDAZOLES,BENZODIAZEPINES,BENZODIAZEPINONES,BENZODIOXOLES,BENZOFURANS,BENZOIC_ACIDS,BENZOTHIADIAZINES,BENZYLISOQUINOLINES,BETA_LACTAMS,BIGUANIDES,BIOGENIC_MONOAMINES,BIPHENYL_COMPOUNDS,BISPHOSPHONATES,BLOOD_COAGULATION_FACTORS,BLOOD_PRODUCTS_AND_SUBSTITUTES,BLOOD_PROTEINS,BORONIC_ACIDS,BRANCHED_CHAIN_AMINO_ACIDS,BRONCHODILATORS,BUTYLENE_GLYCOLS,BUTYRIC_ACIDS,BUTYROPHENONES,CALCIUM_REGULATORS,CALCIUM_SUPPLEMENTS,CAMPTOTHECINS,CAPROATES,CARBAMATES,CARBAPENEMS,CARBAZOLES,CARBOHYDRATES,CARBOXYLIC_ACIDS,CARDIAC_GLYCOSIDES,CARDIOTONICS,CARDIOVASCULAR_THERAPIES,CATECHOLAMINES,CATECHOLAMINES_THERAPEUTIC_USE,CEPHALOSPORINS,CEPHALOSPORINS_DRUG_INTERACTIONS,CEPHALOSPORINS_THERAPEUTIC_USE,CEPHAMYCINS,CEPHEMS,CHEMOPREVENTATIVES,CHEMOPROTECTANTS,CHEMOSENSITISERS,CHLORINATED_HYDROCARBONS,CHLORINATED_STEROIDS,CHLOROBENZENES,CHOLESTENES,CHOLIC_ACIDS,CICLOSPORINS,CLASS_IB_ANTIARRHYTHMICS,CLASS_II_ANTIARRHYTHMICS,CLASS_III_ANTIARRHYTHMICS,CLASS_IV_ANTIARRHYTHMICS,COLCHICUM_ALKALOIDS,CONJUGATE_VACCINES,CONTRAST_MEDIA,CORTICOSTEROIDS,CORTICOSTEROIDS_THERAPEUTIC_USE,COUMARINS,CRESOLS,CYCLIC_ETHERS,CYCLIC_PEPTIDES,CYCLOHEXANECARBOXYLIC_ACIDS,CYCLOHEXANES,CYCLOHEXANOLS,CYCLOHEXENES,CYCLOPARAFFINS,CYCLOPENTANES,CYCLOPHOSPHAMIDES,CYCLOPROPANES,CYSTEINES,CYTOPROTECTIVES,CYTOSTATIC_ANTIBIOTICS,DAUNORUBICINS,DECONGESTANTS,DEOXYRIBONUCLEOSIDES,DIAGNOSTIC_AGENTS,DIAMINO_AMINO_ACIDS,DIAMINOPYRIMIDINES,DIBENZAZEPINES,DIBENZOCYCLOHEPTENES,DIBENZOTHIAZEPINES,DIBENZOXEPINS,DIETHYLAMINES,DIHYDROPYRIDINES,DIHYDROXYCHOLECALCIFEROLS,DIMETHYLAMINES,DISEASE_MODIFYING_ANTIRHEUMATICS,DISINFECTANTS,DITERPENES,DOXORUBICINS,DRUG_WITHDRAWAL_THERAPIES,ECHINOCANDINS,ELECTROLYTES,ENZYMES,EPOXY_COMPOUNDS,ERECTILE_DYSFUNCTION_THERAPIES,ESSENTIAL_AMINO_ACIDS,ESTERS,ESTRADIOL_CONGENERS,ESTRENES,ETHANOLAMINES,ETHYLAMINES,ETHYLENEDIAMINES,EXPECTORANTS,EYE_DISORDER_THERAPIES,FATTY_ACIDS,F

In [24]:
df.columns.to_list()

['Antibiotico',
 'Report Id',
 'Safety Report Name',
 'Record Type',
 'Release Date',
 'Drugs',
 'Adverse Events',
 'Mechanism of Action',
 'Number of Cases',
 'Serious',
 'First Report',
 'Overdose',
 'Drug Interaction',
 'Narrative and comments',
 'Reporter Country',
 'Source Country',
 'Country of publication',
 'Drug Class',
 'Age Group',
 'Route Of Administration',
 'Treatment',
 'Special Situation',
 'References',
 'URL',
 'Drug_Class_list',
 '2_PYRIDINYLMETHYLSULFINYLBENZIMIDAZOLES',
 '2_RING_HETEROCYCLIC_COMPOUNDS',
 '3_RING_HETEROCYCLIC_COMPOUNDS',
 'ABORTIFACIENTS',
 'ACETAMIDES',
 'ACETANILIDES',
 'ACETATES',
 'ACETIC_ACIDS',
 'ACYCLIC_ACIDS',
 'ADENINE_NUCLEOTIDES',
 'ADJUVANTS',
 'ALCOHOLS',
 'ALKALINISING_AGENTS',
 'ALKALOIDS',
 'AMIDES',
 'AMINES',
 'AMINO_ACIDS',
 'AMINOBENZOIC_ACIDS',
 'AMINOBUTYRIC_ACIDS',
 'AMINOCYCLITOLS',
 'AMINOGLYCOSIDES',
 'AMINOPHENOLS',
 'AMINOQUINOLINES',
 'AMINOSALICYLIC_ACIDS',
 'ANAESTHETICS',
 'ANALGESICS',
 'ANDROSTADIENES',
 'ANDROSTANO

In [26]:
df['Age Group'].value_counts()

Age Group
Elderly                  499
Children                 181
Adolescents               78
Infants                   56
Neonates                  29
Fetus                      2
Adolescents, Children      1
Adolescents, Elderly       1
Name: count, dtype: int64

In [27]:
df['Age'].value_counts()

Age
60.00    36
65.00    35
34.00    35
66.00    34
67.00    33
         ..
0.92      1
1.67      1
96.00     1
0.25      1
1.25      1
Name: count, Length: 112, dtype: int64

In [29]:
df["Seizure"].value_counts()

Seizure
0    1850
1     164
Name: count, dtype: int64

In [30]:
df["Serious"].value_counts()

Serious
Yes    1414
No      600
Name: count, dtype: int64